In [1]:
import pandas as pd
import os
import json
import nltk
import string
from nltk.tokenize import TweetTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
reviews = pd.read_pickle('results/tor_reviews.p')

In [3]:
df_rev = pd.DataFrame(reviews)

In [4]:
df_rev = df_rev.head(50) ##test

In [6]:
def tokenize(sent):
    tknzr = TweetTokenizer()
    return tknzr.tokenize(sent.lower())

In [8]:
def get_net_sentiment_score(tokens):
    """
    If you use the VADER sentiment analysis tools, please cite:
    cite: Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.
    """
    # (positive words - negative words) / number of tokens
    sentim_analyzer = SentimentIntensityAnalyzer()
    pos_count = 0
    neg_count = 0
    for token in tokens:
        if sentim_analyzer.polarity_scores(token)['compound'] >= 0.5:
            pos_count += 1
        if sentim_analyzer.polarity_scores(token)['compound'] <= -0.5:
            neg_count += 1
    #print(pos_count, neg_count)
    return (pos_count - neg_count) / len(tokens)
    

In [9]:
punctuations = '!"#$%&\'()*+-/:;<=>?@[\\]^_`{|}~' # string.punctuation excluding .,
def get_punc_count(tokens):
    count = 0
    for token in tokens:
        if token in punctuations:
            count += 1
    return count / len(tokens)

In [10]:
df_rev['review_tokens'] = df_rev['text'].apply(lambda x: tokenize(x))
df_rev = df_rev[['review_id', 'review_tokens']]
df_rev['review_length']= df_rev.review_tokens.apply(lambda x: len(x))
df_rev['sent_score'] = df_rev.review_tokens.apply(lambda x: get_net_sentiment_score(x))
df_rev['get_punc_count'] = df_rev.review_tokens.apply(lambda x: get_punc_count(x))

In [11]:
df_rev

,review_id,review_tokens,review_length,sent_score,get_punc_count
0,kS4hrhEScwB9V5JATYjvVQ,"[sansotei, serves, some, top, notch, ramen, .,...",109,0.000000,0.009174
1,U04YgYr4PiGUxB46J4xapg,"[limited, vegetarian, options, ., ordered, gre...",36,0.000000,0.055556
2,YDJDfKnx6VpMMo4EBxycGg,"[non-existened, service, !, the, waiter, did, ...",96,0.010417,0.072917
3,2Hk7DNwu3rb2jKHaFfPyCA,"[it, might, be, a, bit, unfair, to, review, gl...",709,0.008463,0.012694
4,D2TcNaPqXxDGQ6T0n-vHXw,"[i, have, been, itching, to, get, to, origin, ...",936,0.016026,0.022436
5,Vtd6Tqm7YMfRTBFLtZIoOQ,"[alright, ,, alright, ,, so, as, it, turns, ou...",616,0.003247,0.022727
6,ZOBEhxOeMnncqIJFkGumRA,"[away, from, the, hubbub, of, the, annex, and,...",658,0.003040,0.010638
7,fWdhtaQld99jXB-N9yu1tg,"[if, you've, ever, driven, by, high, park, on,...",240,0.004167,0.016667
8,t98usw-bvob9JqMJPvvnsw,"[uncle, betty's, is, a, family, foodie, heaven...",608,0.014803,0.018092
9,0RqkBJegJpH7thxPTL30NA,"[i, am, the, first, to, admit, that, i, don't,...",736,0.014946,0.019022
